# Fetching Weather Data and Uploading to AWS S3

This guide provides steps for fetching weather data using the OpenWeatherMap API and then uploading the data to an Amazon S3 bucket using Python.

## Step 1: Obtain an API Key from OpenWeatherMap

1. Register on the [OpenWeatherMap website](https://openweathermap.org/) and create an account.
2. Find and copy the API key from your account dashboard.

## Step 2: Write Python Function to Fetch Weather Data

In [1]:
import requests

def get_weather_data(city, api_key):
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {'q': city, 'appid': api_key}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch weather data")

## Step 3: Set Up AWS Credentials for S3 Access
* Install Boto3 using pip install boto3.
* Configure AWS credentials (AWS Access Key ID and Secret Access Key).
* Created S3 bucket using boto3.

In [2]:
!pip install boto3

### Created Bucket in S3 named as "group-5-sec-2-nikkirastogi-lab-3"

In [3]:
import logging
import boto3
from botocore.exceptions import ClientError

def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region. If a region is not specified,
    the bucket is created in the S3 default region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-east-2'
    :return: True if bucket created, else False
    """
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

# Attempt to create the bucket
bucket_name = 'group-5-sec-2-nikkirastogi-lab-3'  # Ensure this is globally unique and all lowercase
region = 'us-east-2'  # Specify your region

if create_bucket(bucket_name, region):
    print(f'Bucket "{bucket_name}" created successfully.')
else:
    print(f'Failed to create bucket "{bucket_name}". Please check the logs for more details.')


Bucket "group-5-sec-2-nikkirastogi-lab-3" created successfully.


## Step 4: Write Python Function to Upload Data to S3


In [4]:
import json

def upload_to_s3(bucket_name, file_name, data):
    s3 = boto3.client('s3')
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=json.dumps(data))

## Step 5: Combine the Functions in a Script


In [5]:
# Main execution
api_key = "17dcac13e2420593d682db6389f9abfb"   # API key
city = "London"  # desired city
bucket_name = "group-5-sec-2-nikkirastogi-lab-3"  # S3 bucket name
file_name = "weather_data.json"

try:
    weather_data = get_weather_data(city, api_key)
    upload_to_s3(bucket_name, file_name, weather_data)
    print("Data uploaded successfully to S3")
except Exception as e:
    print(f"An error occurred: {e}")

Data uploaded successfully to S3


### Check data is uploaded or not

In [6]:
# Example usage
api_key = "17dcac13e2420593d682db6389f9abfb"  # actual API key
city = "London"
try:
    weather_data = get_weather_data(city, api_key)
    print(weather_data)
except Exception as e:
    print(f"An error occurred: {e}")

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 286.03, 'feels_like': 285.31, 'temp_min': 284.79, 'temp_max': 286.91, 'pressure': 1007, 'humidity': 74}, 'visibility': 10000, 'wind': {'speed': 8.94, 'deg': 250, 'gust': 13.41}, 'clouds': {'all': 40}, 'dt': 1712476077, 'sys': {'type': 2, 'id': 2075535, 'country': 'GB', 'sunrise': 1712467283, 'sunset': 1712515396}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


# Assignment: Groups to Convert JSON to CSV

Step 1. Take the JSON output and convert it to a Dataframe using pandas

In [7]:
import pandas as pd

# Convert to DataFrame
def flatten_json(json_obj, prefix=''):
    """
    Flatten a JSON object into a dictionary with flattened keys.

    Args:
        json_obj (dict): The JSON object to flatten.
        prefix (str): Prefix to prepend to flattened keys.

    Returns:
        dict: Flattened dictionary.
    """
    flat_dict = {}  # Initialize an empty dictionary to store flattened key-value pairs
    for key, value in json_obj.items():  
        if isinstance(value, dict):  
            flat_dict.update(flatten_json(value, prefix + key + '_'))  
        elif isinstance(value, list):  
            for i, item in enumerate(value):  
                flat_dict.update(flatten_json(item, prefix + key + '_'))  
        else:  
            flat_dict[prefix + key] = value  # Add the key-value pair to the flattened dictionary

    return flat_dict  # Return the flattened dictionary

In [8]:
# Flatten the JSON-like data
dict_csv = flatten_json(weather_data)

# Create DataFrame from flattened dictionary
df = pd.DataFrame([dict_csv])

# Print DataFrame
df

,coord_lon,coord_lat,weather_id,weather_main,weather_description,weather_icon,base,main_temp,main_feels_like,main_temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod
0,-0.1257,51.5085,802,Clouds,scattered clouds,03d,stations,286.03,285.31,284.79,...,1712476077,2,2075535,GB,1712467283,1712515396,3600,2643743,London,200


##### Step 2. Now upload the CSV file to the 'group-5-sec-2-nikkirastogi-lab-3' S3 bucket in the cloud with the following naming convention: <group-5-sec-2-nikkirastogi-lab-3>_weather_date_london_<datetimestamp>.csv

In [9]:
from datetime import datetime

# Create a timestamp for the filename
timestamp = datetime.utcfromtimestamp(weather_data['dt']).strftime('%Y-%m-%d_%H%M%S')
filename = f"group_5-sec_2_nikkirastogi_weather_date_london_{timestamp}.csv"  # actual group name

# Save to CSV
df.to_csv(filename, index=False)

# Now, for uploading to S3, ensure you have boto3 installed and configured
def upload_csv_to_s3(bucket_name, file_name, object_name=None):
    if object_name is None:
        object_name = file_name
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(file_name, bucket_name, object_name)

# Upload the file
try:
    bucket_name = 'group-5-sec-2-nikkirastogi-lab-3'  # S3 bucket name
    upload_csv_to_s3(bucket_name, filename)
    print("Data as csv uploaded successfully to S3")
except Exception as e:
    print(f"An error occurred: {e}")

### END CODE ###

Data as csv uploaded successfully to S3


##### Thank you